# Dependencies

In [ ]:
! pip install git+https://github.com/RalfKellner/fintextanalytics

  Cloning https://github.com/RalfKellner/fintextanalytics to /tmp/pip-req-build-h0z14mf2
  Running command git clone --filter=blob:none --quiet https://github.com/RalfKellner/fintextanalytics /tmp/pip-req-build-h0z14mf2
  Resolved https://github.com/RalfKellner/fintextanalytics to commit ec7aee1e5c635f0240243353b6c7666b4f5570cc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.0/594.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 62.0 MB/s eta 0:00:

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Data Preparation

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

data_path = '/content/drive/MyDrive/10k_reports.csv'
df = pd.read_csv(data_path)

In [ ]:
df['text'] = df['text'].str.replace('>', '').str.strip()

In [ ]:
from ipywidgets import interact

pd.options.plotting.backend = 'plotly'

In [ ]:
@interact
def filter_company(company = list(df['company'].unique())):
  return df[df.company == company].head()

interactive(children=(Dropdown(description='company', options=('SOUTHERN CO', 'NORTHROP GRUMMAN CORP /DE/'), v…

In [ ]:
df['company'].unique()

array(['SOUTHERN CO', 'NORTHROP GRUMMAN CORP /DE/'], dtype=object)

In [ ]:
df = df.loc[df['company'] == "NORTHROP GRUMMAN CORP /DE/"]

> **Northrop Grumman Corporation** is an American multinational aerospace and defense technology company. With 95,000 employees and an annual revenue in excess of $30 billion, it is one of the world's largest weapons manufacturers and military technology providers.

In [ ]:
sample_text = df['text'].head(20)
print(sample_text)

65    Item 1A. Risk Factors Our consolidated financi...
66    Item 1B. Unresolved Staff Comments None. FORWA...
67    Item 3. Legal Proceedings We have provided inf...
68    Item 7. Management’s Discussion and Analysis o...
69    Item 7A. Quantitative and Qualitative Disclosu...
70    Item 1A. Risk Factors Our consolidated financi...
71    Item 1B. Unresolved Staff Comments None. -24- ...
72    Item 3. Legal Proceedings We have provided inf...
73    Item 7. Management’s Discussion and Analysis o...
74    Item 7A. Quantitative and Qualitative Disclosu...
75    Item 1A. Risk Factors Our consolidated financi...
76    Item 1B. Unresolved Staff Comments None. FORWA...
77    Item 3. Legal Proceedings We have provided inf...
78    Item 7. Management’s Discussion and Analysis o...
79    Item 7A. Quantitative and Qualitative Disclosu...
80                              Item 1A. Risk Factors 6
81    Item 1B. Unresolved Staff Comments 19 Item 2. ...
82    Item 7. Management’s Discussion and Analys

In [ ]:
from fintextanalytics.utils import report2sentences

# use the report2sentences function to iterate through each item, split into sentences and preprocess text
initialized = False
for idx, row in df.iterrows():
    if initialized:
        report_sentences_prep_tmp = report2sentences(str(row['text']), idx_start=report_sentences_preprocessed.index[-1] + 1, item = row['item'], accDate = row['accDate'], company = row['company'])
        report_sentences_preprocessed = pd.concat((report_sentences_preprocessed, report_sentences_prep_tmp), axis = 0)
    else:
        report_sentences_prep_tmp = report2sentences(str(row['text']), item = row['item'], accDate = row['accDate'], company = row['company'])
        report_sentences_preprocessed = report_sentences_prep_tmp.copy()
        initialized = True

# a mapping between docid (sentences which are used for search) and raw id
docid2idx = dict()
for i, idx in enumerate(report_sentences_preprocessed[report_sentences_preprocessed.use_doc].index):
    docid2idx[i] = idx


In [ ]:
from fintextanalytics.embeddingmodels import EmbeddingAnalyzer

embed_analyzer = EmbeddingAnalyzer()

# get the preprocessed text from all documents we want to use
docs = report_sentences_preprocessed[report_sentences_preprocessed.use_doc].prep_text
# use the embed_analzyer doc2vec model to infer document vectors for each document
doc_vecs = [embed_analyzer.doc_model.infer_vector(doc.split(',')) for doc in docs]
# l2 normalize corresponding vectors
doc_vecs = embed_analyzer._validate_normed_vectors(doc_vecs)


In sum 49 words from the default word lists could not be found in the Word2Vec vocabulary. These words are deleted from the word list:
{'educates', 'lesbians', 'homosexual', 'lobbyist', 'grandchildren', 'lesbian', 'genders', 'eeo', 'lgbt', 'eicc', 'minorities', 'ethic', 'ethnicities', 'parachutes', 'grandparent', 'ethnicity', 'stepchildren', 'ruggie', 'ungc', 'proponent', 'donating', 'vestsballot', 'donates', 'siblings', 'motivates', 'vacancyappreciation', 'perquisites', 'ethnically', 'webpage', 'stepparents', 'bisexual', 'honesty', 'householding', 'gays', 'spousal', 'nondiscrimination', 'marriage', 'interlocks', 'cobc', 'nieces', 'humanity', 'transgender', 'discriminated', 'nephews', 'gay', 'ilo', 'grandparents', 'childbirth', 'ballots'}
Vectors seem not have unit length...the vectors have been l2-normalized.


In [ ]:
@interact
def date(accDate = list(report_sentences_preprocessed['accDate'].unique())):
  return report_sentences_preprocessed[report_sentences_preprocessed.accDate == accDate].head(11)

interactive(children=(Dropdown(description='accDate', options=('2023-01-26T08:32:09.000Z', '2022-01-27T06:39:1…



---



# Task 1: *find_close_docs_to_words method*




> Environmental, social, and corporate governance, also known as **environmental, social, governance,** is an approach to investing that recommends taking environmental issues, social issues and governance issues into account when deciding which companies to invest in.



###ewords

Analyzing the environmental domain of a company's 10-K report unveils its environmental practices, sustainability initiatives, and commitment to a clean and green future. It covers areas such as climate change, renewable energy, emissions reduction, biodiversity conservation, waste management, and pollution control. Stakeholders can gain insights into the company's environmental performance, its adherence to regulations, and its efforts towards sustainable development, aiding informed decision-making and assessing its environmental responsibility.

In [ ]:
# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, wordlist = 'ewords')

unique_topn_docs = topn_docs.loc[:, 0].unique().tolist()
#unique_topn_docs.sort()
original_topn_docs_ids = [docid2idx[id] for id in unique_topn_docs]

for idx, row in report_sentences_preprocessed.loc[original_topn_docs_ids, :].iterrows():
  print('-'*50)
  print(row['raw_text'])
  print('-'*50)

--------------------------------------------------
Environmental laws and regulations can impose substantial fines
    and criminal sanctions for violations, and may require the
    installation of costly pollution control equipment or
    operational changes to limit pollution emissions or discharges
     and/or 
    decrease the likelihood of accidental hazardous substance
    releases.
--------------------------------------------------
--------------------------------------------------
Our operations are subject to and affected by a variety of federal, state, local and foreign environmental laws and regulations.
--------------------------------------------------
--------------------------------------------------
Some of our facilities are, for example, engaged in manufacturing processes that produce greenhouse gas emissions, including carbon dioxide, or rely on products from others that do so.
--------------------------------------------------
---------------------------------------

The extracted sentences touch on various topics such as environmental laws and regulations, greenhouse gas emissions, climate change, sustainability goals, and compliance with environmental regulations. It appears that they have been effective in finding documents that are close in meaning to the specified words (ewords).

1. "Environmental laws and regulations can impose substantial fines and criminal sanctions for violations, and may require the installation of costly pollution control equipment or operational changes to limit pollution emissions or discharges."
2. "We also incur, and expect to continue to incur, costs to comply with current federal and state environmental laws and regulations related to the cleanup of pollutants previously released into the environment."
3. "Our operations are subject to and affected by a variety of federal, state, local and foreign environmental laws and regulations."
4. "We have worked for years to reduce our reliance on fossil-based energy sources, to decrease our greenhouse gas emissions, to reduce our consumption of water and production of waste, and to ensure our compliance with environmental regulations where we operate, enhancing our record of environmental sustainability."
5. "Increased worldwide focus on climate change has led to legislative and regulatory efforts to combat both potential causes and adverse impacts of climate change, including regulation of greenhouse gas emissions."

These sentences effectively convey Northrop Grumman's awareness of and commitment to environmental sustainability, compliance with environmental laws and regulations, and efforts to reduce their ecological footprint. They demonstrate the model's ability to generate relevant and meaningful content related to the specified ESG principles.


In [ ]:
topn_doc_scores

,0,1,2,3,4
clean,0.385935,0.352525,0.352092,0.352073,0.352070
environmental,0.375831,0.365903,0.364379,0.357582,0.346692
epa,0.318562,0.312533,0.264497,0.263349,0.260568
sustainability,0.406016,0.358592,0.353260,0.351042,0.343538
climate,0.434064,0.403279,0.395334,0.393638,0.375457
warming,0.448202,0.431846,0.418129,0.409607,0.374969
biofuels,0.238714,0.237059,0.232891,0.226953,0.207259
biofuel,0.242124,0.241079,0.213657,0.211797,0.211122
green,0.178375,0.178058,0.173922,0.173737,0.170190
renewable,0.259028,0.254032,0.204429,0.201336,0.197124


In [ ]:
words = ['greenhouse', 'warming', 'pollution']

# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, words)

# let us take a look at the closest document for the each word
for word, row in topn_docs.iterrows():
    print(f'The context word for the following documents is: {word}')
    original_id = docid2idx[row[0]]
    print('The corresponing sentence with the highest similarity is:')
    print('-'*100)
    print('')
    for x in topn_docs:
      print(x+1,'-', report_sentences_preprocessed.loc[docid2idx[row[x]]].raw_text)
      print('')
    print('-'*100)


Identified user defined word list. Note this list should contain of strings or list of strings. The latter will be merged to one embedding vector by averaging the words.
The context word for the following documents is: greenhouse
The corresponing sentence with the highest similarity is:
----------------------------------------------------------------------------------------------------

1 - New or more stringent laws and regulations related to greenhouse gas emissions and other climate change related concerns have affected and will likely continue to affect us, our suppliers and our customers.

2 - New or more stringent laws and regulations related to greenhouse gas emissions and other climate change related concerns may adversely affect us, our suppliers and our customers.

3 - Increased worldwide focus on climate change has led to legislative and regulatory efforts to combat both potential causes and adverse impacts of climate change, including regulation of greenhouse gas emissions.

In [ ]:
mask = topn_docs[topn_doc_scores > 0]

mask_ids = []
for idx, row in mask.iterrows():
  tmp = row[row.notnull()].tolist()
  if len(tmp) > 0:
    mask_ids = mask_ids + tmp

mask_ids = list(set(mask_ids))
mask_ids = [int(id) for id in mask_ids]
original_mask_ids = [docid2idx[id] for id in mask_ids]

report_sentences_preprocessed.loc[original_mask_ids, :].sort_values('accDate', ascending=False)

,raw_text,prep_text,use_doc,item,accDate,company
245,Increased worldwide focus on climate change ha...,"increased,worldwide,focus,on,climate,change,ha...",True,item1a,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
246,New or more stringent laws and regulations rel...,"new,or,more,stringent,laws,and,regulations,rel...",True,item1a,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
247,"Some of our facilities are, for example, engag...","some,of,our,facilities,are,for,example,engaged...",True,item1a,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
1009,Increased worldwide focus on climate change ha...,"increased,worldwide,focus,on,climate,change,ha...",True,item1a,2022-01-27T06:39:15.000Z,NORTHROP GRUMMAN CORP /DE/
1010,New or more stringent laws and regulations rel...,"new,or,more,stringent,laws,and,regulations,rel...",True,item1a,2022-01-27T06:39:15.000Z,NORTHROP GRUMMAN CORP /DE/
1011,"Some of our facilities are, for example, engag...","some,of,our,facilities,are,for,example,engaged...",True,item1a,2022-01-27T06:39:15.000Z,NORTHROP GRUMMAN CORP /DE/
7765,These laws and regulations may limit our opera...,"these,laws,and,regulations,may,limit,our,opera...",True,item7,2016-02-01T17:14:57.000Z,NORTHROP GRUMMAN CORP /DE/
9403,These laws and regulations may limit our opera...,"these,laws,and,regulations,may,limit,our,opera...",True,item1a,2015-02-02T17:09:25.000Z,NORTHROP GRUMMAN CORP /DE/
9679,These laws and regulations may limit our opera...,"these,laws,and,regulations,may,limit,our,opera...",True,item1a,2014-02-03T16:43:05.000Z,NORTHROP GRUMMAN CORP /DE/
10376,Environmental laws and regulations provide for...,"environmental,laws,and,regulations,provide,for...",True,item1a,2013-02-04T17:42:40.000Z,NORTHROP GRUMMAN CORP /DE/


Sentences related to **pollution** , does not come up after 2016. It seems that company took the required measures. Starting from 2022, concerns related to **greenhouse gas emissions** and **warming** come up. Let's see if company has solutions to this matter.



---



###swords

Analyzing the social domain of a company's 10-K report reveals its commitment to social responsibility, inclusive practices, and ethical conduct. It encompasses aspects such as employee well-being, workplace safety, diversity, philanthropy, education, and support for vulnerable populations. Stakeholders can assess the company's social performance and its impact on society, facilitating informed decision-making and evaluating its dedication to social responsibility.






In [ ]:
# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, wordlist = 'swords')

unique_topn_docs = topn_docs.loc[:, 0].unique().tolist()
#unique_topn_docs.sort()
original_topn_docs_ids = [docid2idx[id] for id in unique_topn_docs]

for idx, row in report_sentences_preprocessed.loc[original_topn_docs_ids, :].iterrows():
  print('-'*50)
  print(row['raw_text'])
  print('-'*50)

--------------------------------------------------
Navigation & Maritime Systems  - delivers products and services to U.S. and international defense, civil and commercial customers supporting smart navigation, shipboard radar surveillance, ship control, machinery control and integrated combat management systems for naval surface ships; high-resolution undersea sensors for mine hunting, situational awareness and other applications; unmanned marine vehicles; shipboard missile and encapsulated payload launch systems, propulsion and power generation systems, nuclear reactor instrumentation and control and acoustic sensors for submarines and aircraft carriers; inertial navigation systems for all domains (air, land, sea, and space); embedded Global Positioning Systems; and postal automation systems.
--------------------------------------------------
--------------------------------------------------
Considerable uncertainty exists regarding how future budget and program decisions will unfold

In terms of generating sentences similar to the wordlist 'swords' of ESG (Environmental, Social, and Governance) principles, the model seems to have performed reasonably well. It has identified sentences that touch upon social aspects such as employee benefits, reputational harm, workforce vaccination, diversity and inclusion objectives, and social and governance practices consistent with values.

1. "A substantial portion of our current and retired employee population is covered by pension and other post-retirement benefit plans."
2. "In the event of an incident, if our customers fail to use our products properly or if our products or services do not operate as intended, we could be subject to reputational harm and potential liabilities."
3. "We have provided paid leave and flexibility for employees to get vaccinated, and strongly encouraged our workforce to take care of themselves and their colleagues."
4. "The company also has a strong record and is deeply committed to maintaining robust social and governance practices consistent with our values."

These sentences highlight important social aspects such as employee benefits, product liability, workforce vaccination support, and commitment to social and governance practices.

In [ ]:
topn_doc_scores

,0,1,2,3,4
citizen,0.259108,0.250111,0.234724,0.232595,0.232265
citizens,0.308560,0.288940,0.261273,0.257501,0.238516
csr,0.341667,0.338404,0.336207,0.327370,0.325008
disabilities,0.252505,0.247797,0.244158,0.237447,0.234313
disability,0.249417,0.241349,0.236564,0.235502,0.234662
...,...,...,...,...,...
hired,0.209501,0.208505,0.205343,0.204873,0.203475
hires,0.299519,0.294501,0.262664,0.261571,0.257610
hiring,0.406495,0.404692,0.379137,0.373630,0.361643
staffing,0.274750,0.222367,0.222000,0.214529,0.207234


In [ ]:
words = ['bargaining', 'unemployment']

# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, words)

print('')
# let us take a look at the closest document for the each word
for word, row in topn_docs.iterrows():
    bold_word = "\033[1m" + word + "\033[0m"
    u_bold_word = "\033[4m" + bold_word + "\033[0m"
    print(f'The context word for the following documents is: {u_bold_word}')
    original_id = docid2idx[row[0]]
    print('The corresponing sentence with the highest similarity is:')
    print('-'*100)
    print('')
    for x in topn_docs:
      print(x+1,'-', report_sentences_preprocessed.loc[docid2idx[row[x]]].raw_text)
      print('')
    print('-'*100)

Identified user defined word list. Note this list should contain of strings or list of strings. The latter will be merged to one embedding vector by averaging the words.

The context word for the following documents is: bargaining
The corresponing sentence with the highest similarity is:
----------------------------------------------------------------------------------------------------

1 - Approximately 20 percent of our 117,100 employees are
    covered by an aggregate of 32 collective bargaining agreements.

2 - Approximately  3,300  of our  65,300  employees are covered by an aggregate of  16  collective bargaining agreements, and we expect to negotiate renewals of two of our collective bargaining agreements in 2014.

3 - Approximately  3,300  of our  68,100  employees are covered by an aggregate of  17  collective bargaining agreements, and we expect to negotiate or re-negotiate renewals for  three  of our collective bargaining agreements in  2013 .

4 - Any such expenses or dela

In [ ]:
words = ['bargaining']

# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, words)

mask = topn_docs[topn_doc_scores > 0]

mask_ids = []
for idx, row in mask.iterrows():
  tmp = row[row.notnull()].tolist()
  if len(tmp) > 0:
    mask_ids = mask_ids + tmp

mask_ids = list(set(mask_ids))
mask_ids = [int(id) for id in mask_ids]
original_mask_ids = [docid2idx[id] for id in mask_ids]

print('')
for idx, row in report_sentences_preprocessed.loc[original_mask_ids, :].sort_values("accDate").iterrows():
  print('-'*150)
  print('')
  print(row['raw_text'])
  print('')
print('-'*150)

Identified user defined word list. Note this list should contain of strings or list of strings. The latter will be merged to one embedding vector by averaging the words.

------------------------------------------------------------------------------------------------------------------------------------------------------

Approximately 20 percent of our 117,100 employees are
    covered by an aggregate of 32 collective bargaining agreements.

------------------------------------------------------------------------------------------------------------------------------------------------------

EMPLOYEE
RELATIONS    We believe that we maintain good relations with our 72,500 employees, of which approximately 3,400 are covered by 17 collective
bargaining agreements.

------------------------------------------------------------------------------------------------------------------------------------------------------

Approximately  3,300  of our  68,100  employees are covered by an aggregate 

In [ ]:
report_sentences_preprocessed.loc[original_mask_ids, :].sort_values("accDate")

,raw_text,prep_text,use_doc,item,accDate,company
12018,"Approximately 20 percent of our 117,100 employ...","approximately,percent,of,our,employees,are,cov...",True,item1a,2011-02-09T12:07:10.000Z,NORTHROP GRUMMAN CORP /DE/
11035,EMPLOYEE\nRELATIONS We believe that we main...,"employee,relations,we,believe,that,we,maintain...",True,item1a,2012-02-08T09:01:52.000Z,NORTHROP GRUMMAN CORP /DE/
10271,"Approximately 3,300 of our 68,100 employee...","approximately,of,our,employees,are,covered,by,...",True,item1a,2013-02-04T17:42:40.000Z,NORTHROP GRUMMAN CORP /DE/
9571,"Approximately 3,300 of our 65,300 employee...","approximately,of,our,employees,are,covered,by,...",True,item1a,2014-02-03T16:43:05.000Z,NORTHROP GRUMMAN CORP /DE/
9574,Any such expenses or delays could adversely af...,"any,such,expenses,or,delays,could,adversely,af...",True,item1a,2014-02-03T16:43:05.000Z,NORTHROP GRUMMAN CORP /DE/


In [ ]:
for txt in report_sentences_preprocessed.loc[12018]:
  print(txt)

Approximately 20 percent of our 117,100 employees are
    covered by an aggregate of 32 collective bargaining agreements.
approximately,percent,of,our,employees,are,covered,by,an,aggregate,of,collective,bargaining,agreements
True
item1a
2011-02-09T12:07:10.000Z
NORTHROP GRUMMAN CORP /DE/


In [ ]:
for txt in report_sentences_preprocessed.loc[11035]:
  print(txt)

EMPLOYEE
RELATIONS    We believe that we maintain good relations with our 72,500 employees, of which approximately 3,400 are covered by 17 collective
bargaining agreements.
employee,relations,we,believe,that,we,maintain,good,relations,with,our,employees,of,which,approximately,are,covered,by,collective,bargaining,agreements
True
item1a
2012-02-08T09:01:52.000Z
NORTHROP GRUMMAN CORP /DE/


In [ ]:
words = ['unemployment']

# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, words)

mask = topn_docs[topn_doc_scores > 0]

mask_ids = []
for idx, row in mask.iterrows():
  tmp = row[row.notnull()].tolist()
  if len(tmp) > 0:
    mask_ids = mask_ids + tmp

mask_ids = list(set(mask_ids))
mask_ids = [int(id) for id in mask_ids]
original_mask_ids = [docid2idx[id] for id in mask_ids]

print('')
for idx, row in report_sentences_preprocessed.loc[original_mask_ids, :].sort_values("accDate").iterrows():
  print('-'*150)
  print('')
  print(row['raw_text'])
  print('')
print('-'*150)

Identified user defined word list. Note this list should contain of strings or list of strings. The latter will be merged to one embedding vector by averaging the words.

------------------------------------------------------------------------------------------------------------------------------------------------------

While the financial markets have begun to stabilize
    and improve in 2009 and 2010, the U.S. and global economies
    continue to struggle as a result of high levels of national debt
    and historic levels of borrowing to support stimulus and
    financial support spending.

------------------------------------------------------------------------------------------------------------------------------------------------------

Political and Economic Environment    The
U.S. and global economies have experienced a period of substantial economic uncertainty and turmoil, with high levels of national debt, and the related financial markets have been characterized by signifi

In [ ]:
report_sentences_preprocessed.loc[original_mask_ids, :].sort_values("accDate", ascending=False)

,raw_text,prep_text,use_doc,item,accDate,company
756,INFLATION RISK The global macroeconomic enviro...,"inflation,risk,the,global,macroeconomic,enviro...",True,item7a,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
76,The global macroeconomic environment has exper...,"the,global,macroeconomic,environment,has,exper...",True,item1a,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
1205,But we continued to see a prolonged impact on ...,"but,we,continued,to,see,prolonged,impact,on,th...",True,item7,2022-01-27T06:39:15.000Z,NORTHROP GRUMMAN CORP /DE/
11446,Political and Economic Environment The\nU.S...,"political,and,economic,environment,the,and,glo...",True,item7,2012-02-08T09:01:52.000Z,NORTHROP GRUMMAN CORP /DE/
12293,While the financial markets have begun to stab...,"while,the,financial,markets,have,begun,to,stab...",True,item7,2011-02-09T12:07:10.000Z,NORTHROP GRUMMAN CORP /DE/


In [ ]:
print(' ')
for txt in report_sentences_preprocessed.loc[746:761].raw_text:
  print(txt)

 
INTEREST RATE RISK We are exposed to interest rate risk on variable-rate short-term credit facilities for which there were no borrowings outstanding at December 31, 2022.
At December 31, 2022, we have $12.9 billion of long-term debt, primarily consisting of fixed-rate debt, with a fair value of approximately $12.1 billion.
The terms of our fixed-rate debt obligations do not generally allow investors to demand payment of these obligations prior to maturity.
Therefore, we do not have significant exposure to interest rate risk for our fixed-rate debt; however, we do have exposure to fair value risk if we repurchase or exchange long-term debt prior to maturity.
Additionally, if we were to refinance our long-term debt, it may be refinanced at higher interest rates.
FOREIGN CURRENCY RISK In certain circumstances, we are exposed to foreign currency risk.
We enter into foreign currency forward contracts to manage a portion of the exchange rate risk related to receipts from customers and paym



---



###gwords

Analyzing the governance domain of a company's 10-K report helps stakeholders understand the company's compliance with regulations, its engagement with government entities, and any legal or regulatory risks it faces. It allows investors, customers, and other stakeholders to assess the company's ability to navigate the legal landscape, manage regulatory challenges.

In [ ]:
# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, wordlist = 'gwords')

unique_topn_docs = topn_docs.loc[:, 0].unique().tolist()
#unique_topn_docs.sort()
original_topn_docs_ids = [docid2idx[id] for id in unique_topn_docs]

for idx, row in report_sentences_preprocessed.loc[original_topn_docs_ids, :].iterrows():
  print('-'*50)
  print(row['raw_text'])
  print('-'*50)

--------------------------------------------------
Subsequent Realignment  – Effective January 1, 2020, the company reorganized its sectors to better align the company’s broad portfolio to serve its customers’ needs.
--------------------------------------------------
--------------------------------------------------
Organization At December 31, 2019, the company was aligned in four operating sectors, which also comprise our reportable segments: Aerospace Systems, Innovation Systems, Mission Systems and Technology Services.
--------------------------------------------------
--------------------------------------------------
Navigation Systems   delivers products and services to defense, civil, and commercial customers supporting situational awareness, inertial
navigation in all domains (air, land, sea, and space), embedded Global Positioning Systems, Identification Friend or Foe (IFF) systems, acoustic sensors, cockpit video monitors, mission computing, and integrated avionics and ele

In terms of generating sentences similar to the 'gwords' of ESG (Environmental, Social, and Governance) principles, the model has performed reasonably well. It has identified sentences that touch upon various aspects of governance, such as organizational structure, compliance with regulations, accounting policies, risk management, and reputation management.

Here are four sentences that exemplify the model's performance in generating sentences related to the 'gwords' wordlist:

1. "Our reputation and our ability to do business may be impacted by the improper conduct of employees, agents, business partners, or joint ventures in which we participate."
2. "CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JUDGMENTS Our consolidated financial statements are prepared in conformity with GAAP, which requires us to make estimates and assumptions about future events that affect the amounts reported in our consolidated financial statements."
3. "New regulations or procurement requirements (including, for example, counterfeit parts and conflict minerals) or changes to current ones could increase our costs and risks and reduce our margins."
4. "Our international business is subject to both U.S. and foreign laws and regulations, including, without limitation, laws and regulations relating to import-export controls, technology transfer restrictions, data privacy and protection, investment, exchange rates and controls, the Foreign Corrupt Practices Act (FCPA) and other anti-corruption laws, the anti-boycott provisions of the U.S."

These sentences highlight important governance-related aspects such as organizational changes, reputation management, accounting policies, and compliance with regulations. They demonstrate the model's ability to generate relevant content related to the specified 'gwords' of ESG principles.

In [ ]:
topn_doc_scores

,0,1,2,3,4
align,0.272332,0.269841,0.264625,0.255078,0.253315
aligned,0.189446,0.183126,0.178109,0.175799,0.173837
aligning,0.289624,0.244124,0.243740,0.241018,0.228175
alignment,0.227087,0.213807,0.205883,0.199377,0.198573
aligns,0.276913,0.272413,0.254998,0.245151,0.242619
...,...,...,...,...,...
notice,0.305429,0.297338,0.293945,0.288448,0.278529
relations,0.372415,0.371344,0.369501,0.363862,0.361892
stakeholder,0.355996,0.351945,0.339136,0.329431,0.315240
stakeholders,0.343990,0.341851,0.340332,0.335035,0.318394


In [ ]:
words = ['conformity']

# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, words)

# let us take a look at the closest document for the each word
for word, row in topn_docs.iterrows():
    print(f'The context word for the following documents is: {word}')
    original_id = docid2idx[row[0]]
    print('The corresponing sentence with the highest similarity is:')
    print('-'*100)
    print('')
    for x in topn_docs:
      print(x+1,'-', report_sentences_preprocessed.loc[docid2idx[row[x]]].raw_text)
      print('')

Identified user defined word list. Note this list should contain of strings or list of strings. The latter will be merged to one embedding vector by averaging the words.
The context word for the following documents is: conformity
The corresponing sentence with the highest similarity is:
----------------------------------------------------------------------------------------------------

1 - CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JUDGMENTS Our consolidated financial statements are prepared in conformity with GAAP, which requires us to make estimates and assumptions about future events that affect the amounts reported in our consolidated financial statements.

2 - CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JUDGMENTS Our consolidated financial statements are prepared in conformity with GAAP, which requires us to make estimates and assumptions about future events that affect the amounts reported in our consolidated financial statements.

3 - CRITICAL ACCOUNTING POLICIES, ESTIMATES, A

In [ ]:
mask = topn_docs[topn_doc_scores > 0]

mask_ids = []
for idx, row in mask.iterrows():
  tmp = row[row.notnull()].tolist()
  if len(tmp) > 0:
    mask_ids = mask_ids + tmp

mask_ids = list(set(mask_ids))
mask_ids = [int(id) for id in mask_ids]
original_mask_ids = [docid2idx[id] for id in mask_ids]

report_sentences_preprocessed.loc[original_mask_ids, :].sort_values('accDate')

,raw_text,prep_text,use_doc,item,accDate,company
9108,"CRITICAL ACCOUNTING POLICIES, ESTIMATES, AND J...","critical,accounting,policies,estimates,and,jud...",True,item7,2015-02-02T17:09:25.000Z,NORTHROP GRUMMAN CORP /DE/
8052,"CRITICAL ACCOUNTING POLICIES, ESTIMATES, AND J...","critical,accounting,policies,estimates,and,jud...",True,item7,2016-02-01T17:14:57.000Z,NORTHROP GRUMMAN CORP /DE/
4743,"CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JU...","critical,accounting,policies,estimates,and,jud...",True,item7,2019-01-31T06:39:46.000Z,NORTHROP GRUMMAN CORP /DE/
1383,"CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JU...","critical,accounting,policies,estimates,and,jud...",True,item7,2022-01-27T06:39:15.000Z,NORTHROP GRUMMAN CORP /DE/
626,"CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JU...","critical,accounting,policies,estimates,and,jud...",True,item7,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/


In [ ]:
print("2019")
print("------------")
for txt in report_sentences_preprocessed.loc[4743:4760].raw_text:
  print(txt)

2019
------------
CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JUDGMENTS Our consolidated financial statements are based on GAAP, which requires us to make estimates and assumptions about future events that affect the amounts reported in our consolidated financial statements.
We employ judgment in making our estimates in consideration of historical experience, currently available information and various other assumptions that we believe to be reasonable under the circumstances.
Actual results could differ from our estimates and assumptions, and any such differences could be material to our consolidated financial statements.
We believe the following accounting policies are critical to the understanding of our consolidated financial statements and require the use of significant management judgment in their application.
For a summary of our significant accounting policies, see Note  1  to the consolidated financial statements.
- 37 - NORTHROP GRUMMAN CORPORATION                          Re

In [ ]:
print("2022")
print("------------")
for txt in report_sentences_preprocessed.loc[1383:1400].raw_text:
  print(txt)

2022
------------
CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JUDGMENTS Our consolidated financial statements are prepared in conformity with GAAP, which requires us to make estimates and assumptions about future events that affect the amounts reported in our consolidated financial statements.
We employ judgment in making our estimates in consideration of historical experience, currently available information and various other assumptions that we believe to be reasonable under the circumstances.
Actual results could differ from our estimates and assumptions, and any such differences could be material to our consolidated financial statements.
We believe the following accounting policies are critical to the understanding of our consolidated financial statements and require the use of significant management judgment in their application.
For a summary of our significant accounting policies, see Note 1 to the consolidated financial statements.
Revenue Recognition Due to the long-term natur

In [ ]:
print("2023")
print("------------")
for txt in report_sentences_preprocessed.loc[626:642].raw_text:
  print(txt)

2023
------------
CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JUDGMENTS Our consolidated financial statements are prepared in conformity with GAAP, which requires us to make estimates and assumptions about future events that affect the amounts reported in our consolidated financial statements.
We employ judgment in making our estimates in consideration of historical experience, currently available information and various other assumptions that we believe to be reasonable under the circumstances.
Actual results could differ from our estimates and assumptions, and any such differences could be material to our consolidated financial statements.
We believe the following accounting policies are critical to the understanding of our consolidated financial statements and require the use of significant management judgment in their application.
For a summary of our significant accounting policies, see Note 1 to the consolidated financial statements.
Revenue Recognition Due to the long-term natur

Based on the information provided in the *CRITICAL ACCOUNTING POLICIES, ESTIMATES AND JUDGMENTS*, it appears that the 2023 report mentions a higher level of volatility compared to 2019 and 2022 reports. The 2023 excerpt specifically states that the volatility of the recent macroeconomic environment has added complexity to the estimation process. It also mentions the impact of inflation and supply chain disruptions on their estimates. These factors suggest a more unpredictable and unstable business environment during the period covered by the 2023 report.

Therefore, based on the above texts, the 2023 year seems to be more volatile compared to both the 2019 and 2022 years.



---



# Task 2: *words of interest analysis*



> The words were chosen to identify potential impacts of war in Ukraine and the increase of inflation on the company's operations. The goal is to uncover any disclosure of risks, challenges, potential opportunities, strategies, or any other relevant details.








In [ ]:
# Word list
wordlist = ['ukraine', ['inflation', 'increase']]

# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, wordlist = wordlist)

print(' ')
topn_doc_scores

Identified user defined word list. Note this list should contain of strings or list of strings. The latter will be merged to one embedding vector by averaging the words.
 


,0,1,2,3,4
ukraine,0.473450,0.423186,0.394563,0.392972,0.386917
inflation increase,0.225628,0.215915,0.200793,0.196719,0.194342


In [ ]:
# let us take a look at the closest documents
print('-'*100)
for word, row in topn_docs.iterrows():
    bold_word = "\033[1m" + word + "\033[0m"
    u_bold_word = "\033[4m" + bold_word + "\033[0m"
    print(f'The context word for the following documents is: {u_bold_word}')
    print('The corresponing sentences, sorted by similarity:')
    print('-'*100)
    print('')
    for x in topn_docs:
      print(x+1,'-', report_sentences_preprocessed.loc[docid2idx[row[x]]].raw_text)
      print('')
    print('-'*100)

----------------------------------------------------------------------------------------------------
The context word for the following documents is: ukraine
The corresponing sentences, sorted by similarity:
----------------------------------------------------------------------------------------------------

1 - However, we do not have sizable business dealings in Russia or Ukraine, and do not anticipate significant adverse impacts from the ongoing conflict.

2 - The conflict in Ukraine has increased global tensions and instability, highlighted threats and increased global demand, as well as further disrupted global supply chains and added costs.

3 - Global Security and Economic Environment The U.S. and its allies continue to face a global security environment of heightened tensions and instability, threats from state and non-state actors as well as terrorist organizations, emerging nuclear tensions, diverse regional security concerns and political instability.

4 - For further inform

In [ ]:
mask = topn_docs[topn_doc_scores > 0]

mask_ids = []
for idx, row in mask.iterrows():
  tmp = row[row.notnull()].tolist()
  if len(tmp) > 0:
    mask_ids = mask_ids + tmp

mask_ids = list(set(mask_ids))
mask_ids = [int(id) for id in mask_ids]
original_mask_ids = [docid2idx[id] for id in mask_ids]

report_sentences_preprocessed.loc[original_mask_ids, :].sort_values('accDate', ascending=False)

,raw_text,prep_text,use_doc,item,accDate,company
81,We (including our suppliers and other partners...,"we,including,our,suppliers,and,other,partners,...",True,item1a,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
478,Current and future requirements related to the...,"current,and,future,requirements,related,to,the...",True,item7,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
439,The conflict in Ukraine has increased global t...,"the,conflict,in,ukraine,has,increased,global,t...",True,item7,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
442,"However, we do not have sizable business deali...","however,we,do,not,have,sizable,business,dealin...",True,item7,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
443,"More broadly, the conflict in Ukraine and thre...","more,broadly,the,conflict,in,ukraine,and,threa...",True,item7,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
1205,But we continued to see a prolonged impact on ...,"but,we,continued,to,see,prolonged,impact,on,th...",True,item7,2022-01-27T06:39:15.000Z,NORTHROP GRUMMAN CORP /DE/
1228,Rising inflation also could lead to increased ...,"rising,inflation,also,could,lead,to,increased,...",True,item7,2022-01-27T06:39:15.000Z,NORTHROP GRUMMAN CORP /DE/
1932,For further information on the potential impac...,"for,further,information,on,the,potential,impac...",True,item7,2021-01-28T06:36:59.000Z,NORTHROP GRUMMAN CORP /DE/
3234,Global Security and Economic Environment The U...,"global,security,and,economic,environment,the,a...",True,item7,2020-01-30T06:39:26.000Z,NORTHROP GRUMMAN CORP /DE/
10220,The President's proposed budget for FY 2013 re...,"the,president,proposed,budget,for,fy,represent...",True,item1a,2013-02-04T17:42:40.000Z,NORTHROP GRUMMAN CORP /DE/


In [ ]:
for txt in report_sentences_preprocessed.loc[437:444].raw_text:
  print(txt)

Our operations and financial performance, as well as demand for our products and services, are impacted by global events, including violence and unrest.
The same is true for our suppliers and other business partners.
The conflict in Ukraine has increased global tensions and instability, highlighted threats and increased global demand, as well as further disrupted global supply chains and added costs.
We have experienced a modest increase in demand for certain of our goods and services directly and indirectly related to the conflict in the Ukraine.
We also have experienced a slight disruption to some of our programs and supply chain, including unanticipated cost growth, as a result of the conflict in Ukraine and economic sanctions.
However, we do not have sizable business dealings in Russia or Ukraine, and do not anticipate significant adverse impacts from the ongoing conflict.
More broadly, the conflict in Ukraine and threats elsewhere have heightened tensions and highlighted security 

In [ ]:
for txt in report_sentences_preprocessed.loc[75:98].raw_text:
  print(txt)

Our business, financial position, results of operations and/or cash flows have been and may continue to be adversely impacted by the global macroeconomic environment.
The global macroeconomic environment has experienced, and continues to experience, extraordinary challenges, including high rates of inflation; widespread disruptions in supply chains; workforce challenges, including labor shortages; and market volatility.
These challenges have, among other things, led to increased costs, labor and supply shortages, and delays and disruption in performance, as well as competing demands for scarce resources.
Those challenges have adversely impacted our customers, our industry, our company, our suppliers and others with whom we do business.
While some aspects of the macroeconomic environment appear to be improving, and we have been able to mitigate some of the challenges (especially with respect to labor shortages), other challenges persist.
We cannot predict the future trajectory of this r

Ukraine-related Sentences: The sentences in this group discuss the conflict in Ukraine and its potential impacts on global tensions, supply chains, security requirements, and demand for defense products. From an investment perspective, these sentences suggest that companies operating in the defense sector might experience increased demand for their products and services due to heightened tensions and security concerns. This could potentially be viewed as a positive factor for the stock prices of defense-related companies.

Inflation-related Sentences: The sentences in this group mention rising inflation, increased interest rates, supply chain disruption, labor shortages, and higher costs. Inflation and its associated challenges can negatively affect companies by increasing their operational expenses and potentially reducing profit margins. If a company is unable to pass on the increased costs to customers, it may face financial pressure. Consequently, these sentences could be seen as potential risk factors that might negatively impact stock prices, especially for companies that are more vulnerable to inflationary pressures.

Based solely on the provided sentences, here is a summary of the potential implications:

1. Adverse Impact of Global Macroeconomic Environment: The company acknowledges that its business, financial position, and operations can be adversely affected by challenges in the global macroeconomic environment. This suggests that the company's performance might be influenced by factors such as high rates of inflation, disruptions in supply chains, labor shortages, market volatility, and increased costs. These challenges could pose risks to the company's financial position and results of operations.

2. Increased Demand for Defense Products and Services: The conflict in Ukraine and global threats have heightened tensions and security requirements worldwide, especially in Europe, the Pacific region, and the U.S. The company has experienced a modest increase in demand for certain goods and services related to the conflict. Additionally, there is an expectation of continued increased demand for defense products and services from allies and partner nations in these regions. This factor could indicate potential opportunities for the company within the defense sector.

3. Disruptions and Unanticipated Costs: The conflict in Ukraine and economic sanctions have caused disruptions to the company's programs and supply chain, leading to unanticipated cost growth. This suggests potential challenges in executing projects efficiently and managing costs effectively.

4. Limited Business Dealings in Russia and Ukraine: The company does not have significant business dealings in Russia or Ukraine and does not anticipate significant adverse impacts from the ongoing conflict. This factor implies that the direct impact of the conflict on the company's financial position and operations might be limited.

5. Increased Competition and Bid Protests: The company operates in highly competitive markets and faces competition from both domestic and international firms. The competitive landscape might affect the company's ability to win new contracts, resulting in reduced revenues and market share. Bid protests can also impact contract execution and earnings.

Considering these factors, it is important to note that stock predictions based solely on these sentences are highly speculative and unreliable. Additional factors, such as financial performance, industry trends, market conditions, and investor sentiment, should be thoroughly analyzed before making any investment decisions.



---



# Task 3:  *fit_topic_model method*

In [ ]:
embed_analyzer.fit_topic_model(doc_vecs)
print('')
print('-'*100)
print('Top 10 words of topic number 1:')
print(embed_analyzer.topic_words[0][:10])
print('-'*100)
print('Top 10 words of topic number 2:')
print(embed_analyzer.topic_words[1][:10])
print('-'*100)
print('...')
print('-'*100)
print(f'Top 10 words of topic number {len(embed_analyzer.topic_words)}:')
print(embed_analyzer.topic_words[-1][:10])
print('-'*100)

Starting to train the dimensionality reduction model...
Umap model for dimensionality reduction has been trained!

Starting to train the clustering model...
HDBSCAN model for clustering has been trained!

Creating topic vectors...
Creating topic vectors finished! 35 topics have been found.
Creating a reduced topic model...
The number of topics has been reduced from 35 to 33.
Topic reduction is finished, the number of reduced topics is 33

----------------------------------------------------------------------------------------------------
Top 10 words of topic number 1:
['intellectual' 'property' 'proprietary' 'rights' 'protect' 'copyrights'
 'copyright' 'infringing' 'infringes' 'secret']
----------------------------------------------------------------------------------------------------
Top 10 words of topic number 2:
['disclaims' 'speak' 'disclaim' 'speaks' 'forward' 'looking' 'hereof'
 'pslra' 'intentions' 'expressly']
-----------------------------------------------------------------

In [ ]:
embed_analyzer.topic_words

array([['intellectual', 'property', 'proprietary', 'rights', 'protect',
        'copyrights', 'copyright', 'infringing', 'infringes', 'secret',
        'parties', 'policing', 'misappropriates', 'licensors',
        'infringement', 'misappropriated', 'licensor', 'nondisclosure',
        'copying', 'patented', 'accuse', 'infringements', 'infringe',
        'misappropriate', 'copyrighted'],
       ['disclaims', 'speak', 'disclaim', 'speaks', 'forward', 'looking',
        'hereof', 'pslra', 'intentions', 'expressly', 'undertakes',
        'undue', 'cautionary', 'cautioned', 'beliefs', 'cautions',
        'rarely', 'revise', 'herein', 'meaning', 'undertake', 'surtaxes',
        'relied', 'disclaimer', 'quantified'],
       ['debarred', 'incarceration', 'recompete', 'relinquishment',
        'reletting', 'sonobuoy', 'ancial', 'fined', 'rebidding',
        'luxiang', 'subcontracts', 'teaming', 'debarment', 'tinuum',
        'eig', 'curtailment', 'wuge', 'cartxpress', 'yuxing',
        'sancti

In [ ]:
topn_topics, topn_topic_scores = embed_analyzer.find_close_topics_to_words(embed_analyzer.topic_vectors, wordlist = 'esgwords')
topn_doc_topics, topn_doc_topic_scores = embed_analyzer.find_close_topics_to_docs(doc_vecs, embed_analyzer.topic_vectors)
topn_topic_docs, topn_topic_doc_scores = embed_analyzer.find_close_docs_to_topics(embed_analyzer.topic_vectors, doc_vecs, topn_docs = 10)



---


The 'esgwords' topic scores demonstrate a strong focus on corporate governance, indicating its significance in the analysis.



---



In [ ]:
topn_topics.loc[:, 0].value_counts()

23    58
11    46
24    44
25    34
14    30
31    27
26    19
19    19
15    18
20    17
13    13
0     12
1     12
10    10
4     10
32    10
3      8
28     7
29     6
8      4
21     4
27     3
18     3
16     3
9      2
5      2
6      2
30     1
Name: 0, dtype: int64

In [ ]:
topn_topic_scores

,0,1,2
clean,0.239466,0.234411,0.181723
environmental,0.378872,0.344852,0.251809
epa,0.186748,0.170698,0.167787
sustainability,0.446932,0.274670,0.144567
climate,0.278794,0.242704,0.165861
...,...,...,...
notice,0.173904,0.135883,0.115092
relations,0.197824,0.182629,0.138194
stakeholder,0.291326,0.218338,0.141199
stakeholders,0.277296,0.217696,0.156552


In [ ]:
doc_idx_start, doc_idx_end = 70, 300

top_topic = topn_doc_topics.loc[doc_idx_start:doc_idx_end, 0].value_counts().index[0]
print('-'*100)
print(' ')
print('Topic words for the topic with the highest count in this report section are:')
print(' ')
print('-'*100)
print(' ')
print(embed_analyzer.topic_words[top_topic])
print(' ')
print('-'*100)
print(' ')
mask = topn_doc_topics.loc[doc_idx_start:doc_idx_end, 0] == top_topic
corresponding_docids = topn_doc_topics.loc[doc_idx_start:doc_idx_end, 0][mask].index
print('Documents which are related in this part of the report read as follows:')
print(' ')
print('-'*100)
print(' ')
for id in corresponding_docids:
    print(report_sentences_preprocessed.loc[docid2idx[id]].raw_text)
    print('-'*100)

----------------------------------------------------------------------------------------------------
 
Topic words for the topic with the highest count in this report section are:
 
----------------------------------------------------------------------------------------------------
 
['unprecedented' 'priorities' 'geopolitical' 'pandemic' 'macroeconomic'
 'turmoil' 'instability' 'economy' 'political' 'macro' 'covid'
 'destabilizing' 'budgetary' 'austerity' 'crises' 'economic' 'stagnant'
 'subside' 'societal' 'economies' 'tensions' 'subsided' 'recession'
 'struggle' 'recessions']
 
----------------------------------------------------------------------------------------------------
 
Documents which are related in this part of the report read as follows:
 
----------------------------------------------------------------------------------------------------
 
The global macroeconomic environment has experienced, and continues to experience, extraordinary challenges, including high rates of

 The provided text indicates that the report section focuses on financial and budget-related matters, highlighting terms like "subcontracts", "struggle", "budget", "cutbacks" and "deficits."



The disclosures within this section cover a range of topics, including macroeconomic challenges, inflationary pressures, supply chain disruptions, cybersecurity threats, and the impact of the COVID-19 pandemic. The report acknowledges the potential adverse effects of these risks on the company's financial position, results of operations, and cash flows. Additionally, it discusses the company's international business risks, including geopolitical factors, laws, and regulations. The importance of environmental, social, and governance matters is also recognized, with a particular emphasis on compliance with relevant requirements. Finally, the report acknowledges the risks posed by natural disasters and climate change, particularly for facilities located in vulnerable areas.



 For investors, the provided sentences offer valuable insights into the potential risks and challenges faced by the company. These factors should be considered when assessing the company's resilience, ability to navigate uncertainties, and potential impacts on financial results and market positioning.



---



# Task 4: *topics of interest analysis*

In [ ]:
wordlist = ['ukraine', 'production', 'demand', 'sales', 'tensions', 'technology', 'defense', 'market', 'contracts', 'procurement',
            'supply', 'export', 'import', 'competition', 'conflict', 'strategy', 'shares', 'prices', 'risks' , 'inflation', 'war']

embed_analyzer.word_list['wordlist'] = wordlist

In [ ]:
embed_analyzer.word_list['wordlist']

['ukraine',
 'production',
 'demand',
 'sales',
 'tensions',
 'technology',
 'defense',
 'market',
 'contracts',
 'procurement',
 'supply',
 'export',
 'import',
 'competition',
 'conflict',
 'strategy',
 'shares',
 'prices',
 'risks',
 'inflation',
 'war']

In [ ]:
## Step 1:
# let us use the ewords word list, you can also define your own, however it should be only a list of strings
wordlist = embed_analyzer.word_list['wordlist']

# this step is optional, the identify_outlier_word method returns two arrays with the same length as the word list
# however, it is increasingly ordered by words which are least similar to the remaining words in the list
outlier_words, outlier_word_scores = embed_analyzer.identify_outlier_word(wordlist)
# update the word list by removing outlier words
wordlist = [word for word in wordlist if not(word in outlier_words[:10])]

## Step 2:
# create a topic vector for the words provided
embed_analyzer.create_wordbased_topic(wordlist, 'spec')

# Step 3:
# look for documents which are most similar to the hand-crafted topic vectors
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_topics(embed_analyzer.manual_topic_vectors['spec'], doc_vecs, 25)
original_ids = [docid2idx[id] for id in topn_docs.values[0]]
x=1
print(' ')
for idx, row in report_sentences_preprocessed.loc[original_ids].iterrows():
    print('-'*100)
    print(x,"-", row.raw_text)
    print('-'*100)
    x = x+1

Vectors seem not have unit length...the vectors have been l2-normalized.
 
----------------------------------------------------------------------------------------------------
1 - Changes in political or economic conditions, including changes in demand, changes in the macroeconomic environment (including inflation and labor and supply chain challenges), changes in defense budgets and/or priorities, changes in export/import restrictions, evolving requirements, or changes in access to critical technology and materials (including metals and components), among others, have adversely affected and could in the future adversely affect the financial stability of our suppliers and/or their ability to perform effectively.
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
2 - Navigation & Maritime Systems  - delivers products and services to U.S. 

Northrop Grumman Corporation, a company in the weapon industry, has been impacted by various factors over the years.

 Changes in political or economic conditions, including demand fluctuations, inflationary pressures, and changes in defense budgets, have adversely affected the company's operations. Current tensions within Congress and the wider U.S. political environment have also influenced defense budgets and government spending. The company acknowledges the unpredictability of the impact on existing and future programs due to changes in threat environments, defense spending levels, and government priorities. Additionally, the adoption of new laws and regulations, environmental impacts, and the dependence on subcontractor and supplier performance have posed challenges. The company also recognizes the global COVID-19 crisis as a significant pressure affecting the U.S. government and governments worldwide.

 Despite these uncertainties, Northrop Grumman has focused on delivering a wide range of products and services in defense and intelligence sectors, including navigation systems, maritime systems, and intelligence sensing solutions. By adapting to geopolitical factors and technological advancements, the company continues to navigate the dynamic landscape of the weapon industry.

The analyzed sentences provide insights into the company's ongoing challenges and responses to evolving conditions in the weapon industry, with a focus on the years 2023 and 2022.

In [ ]:
original_ids

[226,
 7445,
 7,
 4662,
 10979,
 479,
 3790,
 12331,
 10547,
 1512,
 1665,
 10963,
 1043,
 2774,
 10382,
 3578,
 4247,
 2309,
 10553,
 770,
 3743,
 3852,
 2795,
 5388,
 3564]

In [ ]:
report_sentences_preprocessed.loc[original_ids, :].sort_values('accDate', ascending = False)

,raw_text,prep_text,use_doc,item,accDate,company
226,"Changes in political or economic conditions, i...","changes,in,political,or,economic,conditions,in...",True,item1a,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
479,Current tensions within Congress and the wider...,"current,tensions,within,congress,and,the,wider...",True,item7,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
7,"We cannot predict the impact on existing, foll...","we,cannot,predict,the,impact,on,existing,follo...",True,item1a,2023-01-26T08:32:09.000Z,NORTHROP GRUMMAN CORP /DE/
770,"We cannot predict the impact on existing, foll...","we,cannot,predict,the,impact,on,existing,follo...",True,item1a,2022-01-27T06:39:15.000Z,NORTHROP GRUMMAN CORP /DE/
1043,"Changes in laws, regulations, political leader...","changes,in,laws,regulations,political,leadersh...",True,item1a,2022-01-27T06:39:15.000Z,NORTHROP GRUMMAN CORP /DE/
1512,"We cannot predict the impact on existing, foll...","we,cannot,predict,the,impact,on,existing,follo...",True,item1a,2021-01-28T06:36:59.000Z,NORTHROP GRUMMAN CORP /DE/
1665,The global COVID-19 crisis is putting extraord...,"the,global,covid,crisis,is,putting,extraordina...",True,item1a,2021-01-28T06:36:59.000Z,NORTHROP GRUMMAN CORP /DE/
2795,Key programs include Airborne Early Warning & ...,"key,programs,include,airborne,early,warning,co...",True,item7,2020-01-30T06:39:26.000Z,NORTHROP GRUMMAN CORP /DE/
2774,Defense Systems – develops and produces sma...,"defense,systems,develops,and,produces,small,me...",True,item7,2020-01-30T06:39:26.000Z,NORTHROP GRUMMAN CORP /DE/
2309,Key programs include Airborne Early Warning & ...,"key,programs,include,airborne,early,warning,co...",True,item1b,2020-01-30T06:39:26.000Z,NORTHROP GRUMMAN CORP /DE/
